In [1]:
## Table two

In [2]:
from multiprocessing import Pool
import quantileEstimationUnit as qe
from scipy.stats import gamma, lognorm, pareto
import pandas as pd
import numpy as np
import os
import itertools
import sys
import traceback
import dataPreparationUtils as dpu
import optimizationUnit as ou

FILE_DIR = "small"

metaDataDict = {"dataSize": 500,
                "quantitleValue": 0.99,
                "thresholdPercentage": 0.7,
                "alpha": 0.05,
                "gEllipsoidalDimension": 3}

stringToDataModule = {"gamma": gamma,
                      "lognorm": lognorm,
                      "pareto": pareto}


In [3]:
nExperimentReptition = 10
randomSeed = 20220222
# trueValue = dpu.endPointGeneration(
#     gamma, quantitleValue, dpu.dataModuleToDefaultParamDict[gamma])
dataDistributions = ['gamma', 'lognorm', 'pareto']
thresholdPercentages = [0.70]
# served as the target percentage the problem aims to estimate the quantile point from.
quantitleValues = [0.99]
dataSizes = [500]
for dataDistribution, dataSize, quantitleValue, thresholdPercentage in itertools.product(*[dataDistributions, dataSizes, quantitleValues, thresholdPercentages]):
    metaDataDict["dataSize"] = dataSize
    metaDataDict["quantitleValue"] = quantitleValue
    metaDataDict["thresholdPercentage"] = thresholdPercentage
    assert "random_state" not in metaDataDict
    poolParamList = [(dataDistribution, metaDataDict, random_state+randomSeed)
                     for random_state in range(nExperimentReptition)]
    FILE_NAME = ["quantileEstimation"]
    FILE_NAME += ["dataDistribution="+dataDistribution]
    FILE_NAME += [key+"="+str(metaDataDict[key])
                  for key in metaDataDict]
    FILE_NAME += ["randomSeed="+str(randomSeed)]
    FILE_NAME += ["nExperimentReptition="+str(nExperimentReptition)]
    FILE_NAME = '_'.join(FILE_NAME)+".csv"
    FILE_NAME = FILE_NAME.replace("00000000000001","").replace("0000000000001","")
    
    for i in range(len(poolParamList)):
        dataDistribution, metaDataDict, random_state = poolParamList[i]
        metaDataDict["random_state"] = random_state
        dataModule = stringToDataModule[dataDistribution]
        inputData = dpu.RawDataGeneration(dataModule, dpu.dataModuleToDefaultParamDict[dataModule], metaDataDict['dataSize'], metaDataDict['random_state'])
        startQuantilePoint = np.quantile(inputData, thresholdPercentage)
        targetValue = 1-0.99
        lhsPoint = startQuantilePoint
        rhsPoint = np.inf
        midPoint = 2*lhsPoint
        assert False
        ou.OptimizationWithRectangularConstraint(1,
                                                 inputData,
                                                 metaDataDict['thresholdPercentage'],
                                                 metaDataDict['alpha'],
                                                 midPoint, np.inf,
                                                 inputData.size, 7*metaDataDict['random_state']+1)

AssertionError: 

In [15]:
ou.OptimizationWithRectangularConstraint(2,
                                         inputData,
                                         metaDataDict['thresholdPercentage'],
                                         metaDataDict['alpha'],
                                         midPoint*20, np.inf,
                                         inputData.size, 7*metaDataDict['random_state']+1)

0.06919457737022776

In [17]:
threshold = np.quantile(inputData, 0.7)
dataOverThreshold = np.sort(inputData[inputData > threshold])
sizeOverThreshold = np.sum(inputData > threshold)
sizeOnData = inputData.shape[0]

In [20]:
from calibration import etaSpecification, nuSpecification, zOfChiSquare, zOfKolmogorov


In [25]:
z = zOfKolmogorov(alpha=0.05,
                  D_riser_number=1,
                  numMultiThreshold=1)


mu_lb_value = np.maximum(0, (sizeOnData+1-np.arange(
    sizeOnData-sizeOverThreshold+1, sizeOnData+1))/sizeOnData-z/np.sqrt(sizeOnData))
mu_ub_value = np.minimum(1, (sizeOnData-np.arange(
    sizeOnData-sizeOverThreshold+1, sizeOnData+1))/sizeOnData+z/np.sqrt(sizeOnData))

In [26]:
mu_ub_value

array([0.36419686, 0.36219686, 0.36019686, 0.35819686, 0.35619686,
       0.35419686, 0.35219686, 0.35019686, 0.34819686, 0.34619686,
       0.34419686, 0.34219686, 0.34019686, 0.33819686, 0.33619686,
       0.33419686, 0.33219686, 0.33019686, 0.32819686, 0.32619686,
       0.32419686, 0.32219686, 0.32019686, 0.31819686, 0.31619686,
       0.31419686, 0.31219686, 0.31019686, 0.30819686, 0.30619686,
       0.30419686, 0.30219686, 0.30019686, 0.29819686, 0.29619686,
       0.29419686, 0.29219686, 0.29019686, 0.28819686, 0.28619686,
       0.28419686, 0.28219686, 0.28019686, 0.27819686, 0.27619686,
       0.27419686, 0.27219686, 0.27019686, 0.26819686, 0.26619686,
       0.26419686, 0.26219686, 0.26019686, 0.25819686, 0.25619686,
       0.25419686, 0.25219686, 0.25019686, 0.24819686, 0.24619686,
       0.24419686, 0.24219686, 0.24019686, 0.23819686, 0.23619686,
       0.23419686, 0.23219686, 0.23019686, 0.22819686, 0.22619686,
       0.22419686, 0.22219686, 0.22019686, 0.21819686, 0.21619